**Humor Classification**


In [ ]:
!pip install datasets>=2.18.0 transformers>=4.38.2 accelerate>=0.27.2 evaluate
from datasets import load_dataset
import random

all_data = load_dataset("CreativeLang/ColBERT_Humor_Detection").filter(lambda example: len(example["text"])<1024)["train"]
random.seed(1)

# Load our small sample of data for training and analysis
random_train_indices = random.sample(range(len(all_data)), 18000)
train_data = all_data.select(random_train_indices)

random_eval_indices = random.sample(range(len(all_data)), 2000)
eval_data = all_data.select(random_eval_indices)

random_test_indices = random.sample(range(len(all_data)), 2000)
test_data = all_data.select(random_test_indices)


<h1>Classifier 1</h1>
<i>Classifier 1: Rule-based. Write a rule-based classifier with regular expressions</i>

In [ ]:
import re

def regex_classifier(text):
    humor_patterns = [
        r'^(Q:|Question:).*(A:|Answer:)',
        r'[.!?]$',
        r'\bwhat do you\b',
        r'\bwhy did\b',
        r'\bwhy does\b',
        r'who\s+.*'
    ]

    for pattern in humor_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            return 1
    return 0


In [ ]:
for i in range(100):
    sample = eval_data[i]

    text = sample["text"]
    true_label = sample["humor"]

    predicted = regex_classifier(text)

    print(f"=== Sample {i + 1} Analysis ===")
    print("Text: ", text)
    print("True Label: ", "TRUE" if true_label else "FALSE")
    print("Predicted Label: ", "True" if predicted else "False")
    print("========================\n")


=== Sample 1 Analysis ===
Text:  Dalai lama: buddha would have helped persecuted rohingya refugees
True Label:  FALSE
Predicted Label:  False

=== Sample 2 Analysis ===
Text:  Went to a zoo that only had one dog it was a shitzoo
True Label:  TRUE
Predicted Label:  False

=== Sample 3 Analysis ===
Text:  Blake lively beauty evolution: the 'gossip girl' has been 50 shades of blonde (photos)
True Label:  FALSE
Predicted Label:  False

=== Sample 4 Analysis ===
Text:  You are what you eat.. funny, i dont rember eating a sexy beast this morning.
True Label:  TRUE
Predicted Label:  True

=== Sample 5 Analysis ===
Text:  Why this lesbian catholic is supporting the national weekend of prayer for transgender justice
True Label:  FALSE
Predicted Label:  False

=== Sample 6 Analysis ===
Text:  Why does a pencil shave? to look sharp
True Label:  TRUE
Predicted Label:  True

=== Sample 7 Analysis ===
Text:  Abused donkey is rescued in an unlikely place
True Label:  FALSE
Predicted Label:  False

==

In [ ]:
for i in range(20):
  print(train_data[i])


{'text': 'Is it just me or is solipsism just a load of rubbish?', 'humor': True}
{'text': 'Uber needs to do better when it comes to diversity', 'humor': False}
{'text': 'I finally stopped the annoying noise in my car. i just opened the door and shoved her out.', 'humor': True}
{'text': "Transitional age is when during a hot day you don't know what you want  ice cream or beer.", 'humor': True}
{'text': 'They call me the cat whisperer, cause i know exactly what the pussy needs.', 'humor': True}
{'text': "Motorcyclist's horrifying brush with death caught on video", 'humor': False}
{'text': "Most americans aren't planning to talk politics this thanksgiving", 'humor': False}
{'text': 'The beatles drummer, ringo starr, has passed away... ...the band is now known as the beatless.', 'humor': True}
{'text': 'Health care 101: dealing with a medical error', 'humor': False}
{'text': 'Wall street bonuses for top employees dropped 38 percent last year: survey', 'humor': False}
{'text': 'Get your hou

In [ ]:
# evaluation

def recall(true_positives, false_negatives):
  return true_positives / (true_positives + false_negatives)

def precision(true_positives, false_positives):
  return true_positives / (true_positives + false_positives)

def accuracy(true_positives, true_negatives, false_positives, false_negatives):
  return (true_positives + true_negatives) / (true_positives + true_negatives + false_positives + false_negatives)

def f1_score(precision, recall):
  return (2 * precision * recall) / (precision + recall)

def evaluate(data):
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  pos = 0
  neg = 0
  for i in range(len(data)):
    if data[i]["humor"]  == True and regex_classifier(data[i]["text"]) == True:
      tp += 1
    elif data[i]["humor"]  == False and regex_classifier(data[i]["text"]) == False:
      tn += 1
    elif data[i]["humor"]  == False and regex_classifier(data[i]["text"]) == True:
      fn += 1
    else:
      fp += 1

  for i in range(len(data)):
    if data[i]["humor"] == True:
      pos += 1
    else:
      neg += 1

  rec = recall(tp, fn)
  prec = precision(tp, fp)
  acc = accuracy(tp, tn, fp, fn)
  f1 = f1_score(prec, rec)

  pos_per = (pos / (len(data) - 1)) * 100
  neg_per = (neg / (len(data) - 1)) * 100

  print("Positive Joke %:", pos_per)
  print("Negative Joke %:", neg_per)

  print("Recall:", rec)
  print("Precision:", prec)
  print("Accuracy:", acc)
  print("F1 Score:", f1_score(prec, rec))

In [ ]:
# evaluation output
evaluate(test_data)

Positive Joke %: 49.6248124062031
Negative Joke %: 50.42521260630315
Recall: 0.9070904645476773
Precision: 0.7479838709677419
Accuracy: 0.837
F1 Score: 0.8198895027624309


**Design of the Rule-Based Humor Classifier**

This rule-based classifier uses regular expressions (regex) to detect patterns commonly associated with humorous text. It flags a piece of text as humorous (1) if it matches any predefined regex pattern, and non-humorous (0) otherwise. The classifier is designed to look for linguistic cues and structures that are typical of jokes or humorous content.

**Regex Rules Used and Why:**

1.	r'^(Q:|Question:).*(A:|Answer:)' : It matches Q&A formats like “Q: Why did the chicken… A: To get to the other side,” which are classic joke setups.
2.	r'[.!?]$' : It ensures the sentence ends with punctuation, which is typical in well-formed, deliberate jokes (in contrast to casual or incomplete statements).
3.	r'\bwhat do you\b' : Captures joke structures like “What do you call a…” — a common joke opening.
4.	r'\bwhy did\b' and r'\bwhy does\b' : These phrases often signal the setup of a joke or a humorous question.
5.	r'who\s+.*' : Picks up on exaggerated or rhetorical questions like “Who even does that?” which may be used in a humorous or sarcastic tone.

These patterns were chosen because they represent common structure and cues used in jokes, especially in written text like tweets or stand-up scripts. While not foolproof, they give the classifier a simple logic to detect likely humor without needing machine learning.

**Evaluation**
1. For evaluation, we used accuracy, since both true positives and true negatives are important, and the dataset is roughly balanced. The classifier achieved **83.7%** accuracy, which is reasonable for a lightweight, rule-based approach using just six regex rules.


<h1>Classifier 2</h1>
<i>Classifier 2: Logistic Regression + TF-IDF (like we did in class for the imdb dataset)</i>

In [ ]:
# Classifier 2: Logistic Regression + TF-IDF
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer = TfidfVectorizer(max_df=0.5)
vectorizer = TfidfVectorizer(max_df=0.5, ngram_range=(2, 2), min_df=1)
X_train = vectorizer.fit_transform(train_data["text"])
X_test = vectorizer.transform(test_data["text"])

model = LogisticRegression(verbose=1)
model.fit(X_train, train_data["humor"])

predicted_lr = model.predict(X_test)
print(classification_report(test_data["humor"], predicted_lr))

              precision    recall  f1-score   support

       False       0.83      0.92      0.87      1008
        True       0.91      0.80      0.85       992

    accuracy                           0.86      2000
   macro avg       0.87      0.86      0.86      2000
weighted avg       0.87      0.86      0.86      2000



In [ ]:
print(predicted_lr[0])

True


**Design of the Logistic Regression + TF-IDF Classifier**

**Hyperparameters:**
1. With the original hyperparameter we had an accuracy of 91%, however, when we included some hyperparameters such as ngram_range and min_dif there was a 5% decrease in accuracy.

2. We also observed that increasing max_df to 1 drastically decreases the accuracy down to 50% because it allowed overly common, non-informative bigrams into the feature set. These frequent terms added noise and diluted the discriminative power of the model, making it harder to distinguish between humorous and non-humorous text.

**Evaluation Metric**
1. For evaluation metric we decided to focus on the F1-score because it balances precision and recall; both essential for tasks like humor detection where false positives and false negatives carry different consequences.

2. The Logistic Regression + TF-IDF model achieved an accuracy of 86%. This is with the hyperparameters that we included. Compared to the Regex model, which has an accurary of 83.7%, this Logistic Regression + TF-IDF model is 2.3% more accurate. If we kept just the max_df hyperparameter, we would have had an accuracy of 91%. Regardless of the difference in the hyperparameters, the Logistic Regression + TF-IDF model has a higher accuracy than the Regex model.

# <h1>Classifier 3</h1>
<i>Classifier 3: Fine-tune an encoder model (like we did in class on the rotten tomatoes dataset)</i>

In [ ]:
# Classifier 3: Full fine-tuned encoder model
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
import numpy as np
import evaluate

In [ ]:
# Load Model and Tokenizer
model_id = "bert-base-cased"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2) #attach a classification head on top of your model (one layer nn on top of your embeddings)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from datasets import ClassLabel, Value
# Pad to the longest sequence in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
   """Tokenize input data"""
   return tokenizer(examples["text"], truncation=True)

# Tokenize train/test data
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)


tokenized_train = tokenized_train.rename_column("humor", "label")
print(type(tokenized_train[0]["label"]))

tokenized_test = tokenized_test.rename_column("humor", "label")

tokenized_train = tokenized_train.cast_column("label", Value("int64"))
tokenized_test = tokenized_test.cast_column("label", Value("int64"))

print(type(tokenized_train[0]["label"]))

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

<class 'bool'>


Casting the dataset:   0%|          | 0/18000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

<class 'int'>


In [ ]:
from sklearn.metrics import accuracy_score
def compute_metrics(eval_pred):
    """Calculate F1 score"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    load_f1 = evaluate.load("f1")
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]

    """Calculate Accuracy"""
    #accuracy = accuracy_score(labels, predictions)
    load_accuracy = evaluate.load("accuracy")
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]

    return {"f1": f1, "accuracy": accuracy}

In [ ]:
# Training arguments for parameter tuning
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


<ipython-input-9-ebef23302f97>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
print(tokenized_train[0])
# text, humor, input_ids, token_type_ids, attention_mask

{'text': 'Is it just me or is solipsism just a load of rubbish?', 'label': 1, 'input_ids': [101, 2181, 1122, 1198, 1143, 1137, 1110, 1177, 10913, 4863, 1306, 1198, 170, 6829, 1104, 16259, 26652, 136, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
trainer.train()

Step,Training Loss
500,0.124100
1000,0.097600


TrainOutput(global_step=1125, training_loss=0.10620494164360894, metrics={'train_runtime': 137.0008, 'train_samples_per_second': 131.386, 'train_steps_per_second': 8.212, 'total_flos': 288484827761280.0, 'train_loss': 0.10620494164360894, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.08174105733633041,
 'eval_f1': 0.9767911200807265,
 'eval_accuracy': 0.977,
 'eval_runtime': 4.8183,
 'eval_samples_per_second': 415.088,
 'eval_steps_per_second': 25.943,
 'epoch': 1.0}

### Freeze Layers

In [ ]:
# Load Model and Tokenizer
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Print layer names
for name, param in model.named_parameters():
    print(name)

bert.embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight
bert.embeddings.LayerNorm.weight
bert.embeddings.LayerNorm.bias
bert.encoder.layer.0.attention.self.query.weight
bert.encoder.layer.0.attention.self.query.bias
bert.encoder.layer.0.attention.self.key.weight
bert.encoder.layer.0.attention.self.key.bias
bert.encoder.layer.0.attention.self.value.weight
bert.encoder.layer.0.attention.self.value.bias
bert.encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.bias
bert.encoder.layer.0.attention.output.LayerNorm.weight
bert.encoder.layer.0.attention.output.LayerNorm.bias
bert.encoder.layer.0.intermediate.dense.weight
bert.encoder.layer.0.intermediate.dense.bias
bert.encoder.layer.0.output.dense.weight
bert.encoder.layer.0.output.dense.bias
bert.encoder.layer.0.output.LayerNorm.weight
bert.encoder.layer.0.output.LayerNorm.bias
bert.encoder.layer.1.attention.self.query.weight
bert.enc

In [ ]:
for name, param in model.named_parameters():

     # Trainable classification head
     if name.startswith("classifier"):
        param.requires_grad = True

      # Freeze everything else
     else:
        param.requires_grad = False

In [ ]:
# We can check whether the model was correctly updated
for name, param in model.named_parameters():
     print(f"Parameter: {name} ----- {param.requires_grad}")

Parameter: bert.embeddings.word_embeddings.weight ----- False
Parameter: bert.embeddings.position_embeddings.weight ----- False
Parameter: bert.embeddings.token_type_embeddings.weight ----- False
Parameter: bert.embeddings.LayerNorm.weight ----- False
Parameter: bert.embeddings.LayerNorm.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.query.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.query.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.key.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.key.bias ----- False
Parameter: bert.encoder.layer.0.attention.self.value.weight ----- False
Parameter: bert.encoder.layer.0.attention.self.value.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.weight ----- False
Parameter: bert.encoder.layer.0.attention.output.dense.bias ----- False
Parameter: bert.encoder.layer.0.attention.output.LayerNorm.weight ----- False
Parameter: bert.encoder.layer.0.attention.output

In [ ]:
# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)
trainer.train()

<ipython-input-22-71e07c7775f6>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.676600
1000,0.653000


TrainOutput(global_step=1125, training_loss=0.6625599772135417, metrics={'train_runtime': 35.5998, 'train_samples_per_second': 505.621, 'train_steps_per_second': 31.601, 'total_flos': 258712167403200.0, 'train_loss': 0.6625599772135417, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6406593918800354,
 'eval_f1': 0.7517361111111112,
 'eval_accuracy': 0.714,
 'eval_runtime': 5.998,
 'eval_samples_per_second': 333.443,
 'eval_steps_per_second': 20.84,
 'epoch': 1.0}

In [ ]:
# find misclassifications
from numpy import argmax
def misclassifications(data):
  results = trainer.predict(tokenized_test)
  encoder_pred = argmax(results.predictions, axis=-1)
  lr_reg_counter = 0
  encoder_lr_counter = 0
  encoder_counter = 0

  print("Three sentences that the LR model gets correct and the regex model gets incorrect:")
  for i in range(len(data)):
    if (data[i]["humor"]  == True and predicted_lr[i] == True and regex_classifier(data[i]["text"]) == False) or (data[i]["humor"]  == False and predicted_lr[i] == False and regex_classifier(data[i]["text"]) == True):
      lr_reg_counter += 1
      print("Data:", data[i], ", LR:", predicted_lr[i], ", Regex:", regex_classifier(data[i]["text"]))

    if lr_reg_counter == 3:
      break

  print("Three sentences that the encoder model gets correct and the LR model gets incorrect:")
  for i in range(len(data)):
    if (data[i]["humor"]  == True and encoder_pred[i] == True and predicted_lr[i] == False) or (data[i]["humor"]  == False and encoder_pred[i] == False and predicted_lr[i] == True):
      encoder_lr_counter += 1
      print("Data:", data[i], ", Encoder:", encoder_pred[i], ", LR:", predicted_lr[i])

    if encoder_lr_counter == 3:
      break

  print("Three sentences that our best encoder model gets incorrect:")
  for i in range(len(data)):
    if (data[i]["humor"]  == True and encoder_pred[i] == False) or (data[i]["humor"]  == False and encoder_pred[i] == True):
      encoder_counter += 1
      print("Data:", data[i], ", Encoder:", encoder_pred[i])
    if encoder_counter == 3:
      break


In [ ]:
misclassifications(test_data)

Three sentences that the LR model gets correct and the regex model gets incorrect:
Data: {'text': 'How did cosby fuck up his phone? he put it on sleep mode', 'humor': True} , LR: True , Regex: 0
Data: {'text': "Why don't elephants smoke? they can't fit their butts in the ashtray", 'humor': True} , LR: True , Regex: 0
Data: {'text': 'Should police be allowed to keep property without a criminal conviction?', 'humor': False} , LR: False , Regex: 1
Three sentences that the encoder model gets correct and the LR model gets incorrect:
Data: {'text': "It's complicated relationship status = someone cheated but we signed a lease.", 'humor': True} , Encoder: 1 , LR: False
Data: {'text': 'Bought a sled on sale in boston got a real tobahgain.', 'humor': True} , Encoder: 1 , LR: False
Data: {'text': "Plan ahead - it wasn't raining when noah built the ark.", 'humor': True} , Encoder: 1 , LR: False
Three sentences that our best encoder model gets incorrect:
Data: {'text': "Nobody's going to see a stic

In [ ]:
from numpy import argmax
def misclass(data):
  results = trainer.predict(tokenized_test)
  encoder_pred = argmax(results.predictions, axis=-1)
  for i in range(len(data)):
    if (data[i]["humor"]  == True and encoder_pred[i] == False) or (data[i]["humor"]  == False and encoder_pred[i] == True):
      print("Data:", data[i], ", Encoder:", encoder_pred[i])

misclass(test_data)

Data: {'text': 'If only wars could be stopped this way...', 'humor': False} , Encoder: 1
Data: {'text': 'Do lady dolphins ever get tattoos of 19-yr-old community college students?', 'humor': True} , Encoder: 0
Data: {'text': 'Miss piggy can be a such a hamful to deal with', 'humor': True} , Encoder: 0
Data: {'text': 'Live, laugh, love, dress up like a clown and wander around the woods at night', 'humor': True} , Encoder: 0
Data: {'text': '5 years ago, a man with a gun slaughtered 20 children. and we have done little about it.', 'humor': False} , Encoder: 1
Data: {'text': "Here's my review of the sun: one star.", 'humor': True} , Encoder: 0
Data: {'text': 'What are some of the best comments on threads?', 'humor': True} , Encoder: 0
Data: {'text': 'Identity theft and credit card fraud... who is really at risk?', 'humor': False} , Encoder: 1
Data: {'text': 'The person that invented toilet paper is easily the most under-appreciated hero in human history', 'humor': True} , Encoder: 0
Data: 

## **Error Analysis**

*For each set of examples, write your best analysis (20-50 words) of why you think the model got it wrong. Your analysis should explore if there's a general pattern to the type of sentences each classifier gets wrong on this task.*


1. **3 examples of sentences from eval that Logistic Regression+TF-IDF gets right but regex rules gets wrong.**

Sentence 1:
“How did Cosby fuck up his phone? he put it on sleep mode”

**Analysis:**
The regex model misses this joke because it doesn't match any predefined patterns. It lacks understanding of wordplay or cultural references. In contrast, the TF-IDF model likely recognizes humor patterns in the phrasing or keyword associations like “fuck up,” “phone,” and “sleep mode.”

Sentence 2:
“Why don't elephants smoke? they can't fit their butts in the ashtray”

**Analysis:**
Although the joke starts with “Why don't…”, which resembles a pattern the regex is designed for, it likely fails due to strict spacing or token structure. Logistic Regression handles this better by learning from similar bi-gram patterns like “elephants smoke” or “butts ashtray.”

Sentence 3:
“Should police be allowed to keep property without a criminal conviction?”

**Analysis:**
The regex wrongly classifies this as humorous due to the “should [subject]” format resembling a joke pattern. However, the sentence is serious. Logistic Regression correctly identifies the tone as non-humorous by considering word context like “police,” “conviction,” and “property.”



2. **3 examples of sentences from eval that encoder gets right but Logistic Regression gets wrong.**

Sentence 1:
"It's complicated relationship status = someone cheated but we signed a lease."

**Analysis:**
The Encoder model classified the text as humorous because it understands the overall sentence structure and underlying contradiction. It captures the emotional implication of “cheated” versus the mundane “lease”, which creates comedic contrast.
The Logistic Regression classified the text as not humorous because it only sees surface-level bigrams like “signed lease” or “relationship status”, which aren’t humorous on their own and are often found in serious or neutral contexts. Unlike the Encoder model, it can’t understand the deeper irony that comes from connecting different parts of the text.

Sentence 2:'Bought a sled on sale in boston got a real tobahgain.'

**Analysis:**
This text was classified as humor by the Encoder model because it has been trained on a wide range of language and can understand that “tobahgain” is a funny misspelling based on the context. Its ability to understand meaning and context helps it spot the pun and recognize it as humor.
The Logistic Regression classified the text as not humorous because it only looks at simple word patterns and how often words appear together. It doesn’t understand things like context, wordplay, or language differences. The model doesn’t recognize the Boston accent pun (“tobahgain” for “bargain”) or the reference to “toboggan” as a type of sled, missing the irony in the joke. Since “tobahgain” is an uncommon spelling not seen often in its training data, the Logistic Regression model failed to recognize it as a humorous variation.

Sentence 3:"Plan ahead - it wasn't raining when noah built the ark."

**Analysis:**
This text was classified as humor by the Encoder model because the model was likely familiar with the story of Noah’s Ark. This helps it pick up on the idea of planning ahead for something that seems unlikely or far off, and it understands the humor in how that serious preparation is described in a casual or understated way. The Logistic Regression classified the text as not humorous because it likely does not recognize “Noah’s ark” as a well-known story about preparing for a future disaster. It might see “raining” and “built” as separate events without the necessary background knowledge. The model treats “plan ahead”, “noah built”, or “raining when” as isolated bigrams, none of which seem funny without the full background


3. **3 examples of sentences from eval that your best encoder model still gets wrong.**

Sentence 1:
"Nobody's going to see a sticker on a telephone pole and then become a fan of your band."

**Analysis:**
The encoder classifies this sentence incorrectly. It classifies it as not a joke when it is in fact a joke. Since the humor is dry, observational humor, the model is struggling to identify it as a joke. Also it does not follow the typical structure of a joke, making it more challenging for the model to classify correctly.

Sentence 2:
"Talking to marco rubio: a scripted candidate suddenly gets chatty"

**Analysis:**
The encoder classifies this sentence incorrectly. It classifies it as a joke when it is not a joke. With the use of the colon, the structure is similar to some jokes, causing the misclassification. Also, there is irony that the encoder is viewing as humourous, and thus misclassifying it as a joke.

Sentence 3:
"Perfect house, perfect spouse? how finding your dream home is like dating"

**Analysis:**
The encoder classifies this sentence incorrectly. It classifies it as a joke when it is not a joke. The structure is question-answer, which is typical of a joke, and it compares unrelated domains which is also common in humor, causing the encoder to misclassify it as a joke.




<h1>Bonus:</h1>

Bonus (4 points to overall grade): *Zero-shot and few-shot evaluate a generative LM model on the test dataset, and achieve an accuracy greater than 90%.*

In [ ]:
!pip install -q transformers accelerate bitsandbytes datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model_id = "MBZUAI/LaMini-Flan-T5-783M"


tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/860 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
FEW_SHOT = """
Instruction: Is the following text a joke? Respond only with "Yes" or "No".

Text: "Why don’t skeletons fight each other? They don’t have the guts."
Answer: Yes

Text: "The quarterly update will be shared tomorrow."
Answer: No

Text: "I'm on a seafood diet. I see food and I eat it!"
Answer: Yes

Text: "The system will be offline for maintenance at midnight."
Answer: No

Text: "What do you call fake spaghetti? An impasta!"
Answer: Yes

Text: "Please sign the attendance sheet at the front desk."
Answer: No
"""


In [ ]:
def make_few_shot_prompt(text):
    return f"""{FEW_SHOT}

Text: "{text}"
Answer:"""

def make_zero_shot_prompt(text):
    return f"""Instruction: Is the following text a joke? Respond only with "Yes" or "No".
Text: "{text}"
Answer:"""



In [ ]:
def extract_yes_no_only(output_text):
    output_text = output_text.lower()
    if "yes" in output_text:
        return 1
    elif "no" in output_text:
        return 0
    return 0  # fallback to No

def mini_flan_few_shot_predict(text):
    prompt = make_few_shot_prompt(text)
    result = generator(
        prompt,
        max_new_tokens=5,
        temperature=0.0,
        top_p=0.9,
        do_sample=False
    )[0]["generated_text"]
    return extract_yes_no_only(result)

def mini_flan_zero_shot_predict(text):
    prompt = make_zero_shot_prompt(text)
    result = generator(
        prompt,
        max_new_tokens=5,
        temperature=0.0,
        top_p=0.9,
        do_sample=False
    )[0]["generated_text"]
    return extract_yes_no_only(result)



In [ ]:
def evaluate_model(predict_func, data,title="Model"):
    correct = 0
    num = len(data)
    for i in range(num):
        text = data[i]["text"]
        true = data[i]["humor"]
        pred = predict_func(text)
        if pred == true:
            correct += 1
        #print(f"[{i+1}] Pred: {pred} | True: {true} | Text: {text[:50]}...")

    acc = correct / num
    print(f"\n {title} Accuracy on {num} samples: {acc * 100:.2f}%")
    return acc


In [ ]:
evaluate_model(mini_flan_zero_shot_predict, test_data, title="LaMini-Flan-T5 Zero-Shot")
evaluate_model(mini_flan_few_shot_predict, test_data, title="LaMini-Flan-T5 Few-Shot")



/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



 LaMini-Flan-T5 Zero-Shot Accuracy on 2000 samples: 80.20%

 LaMini-Flan-T5 Few-Shot Accuracy on 2000 samples: 74.65%


0.7465

Bonus (1 point to overall grade): *Achieve an accuracy >= 98% on this task with your encoder model.*

In [ ]:
# Load Model and Tokenizer
model_id = "distilbert/distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2) #attach a classification head on top of your model (one layer nn on top of your embeddings)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [ ]:
from datasets import ClassLabel, Value
# Pad to the longest sequence in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
   """Tokenize input data"""
   return tokenizer(examples["text"], truncation=True)

# Tokenize train/test data
tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_test = test_data.map(preprocess_function, batched=True)


tokenized_train = tokenized_train.rename_column("humor", "label")
print(type(tokenized_train[0]["label"]))

tokenized_test = tokenized_test.rename_column("humor", "label")

tokenized_train = tokenized_train.cast_column("label", Value("int64"))
tokenized_test = tokenized_test.cast_column("label", Value("int64"))

print(type(tokenized_train[0]["label"]))

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

<class 'bool'>


Casting the dataset:   0%|          | 0/18000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

<class 'int'>


In [ ]:
# Training arguments for parameter tuning
training_args = TrainingArguments(
   "model",
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   report_to="none"
)

# Trainer which executes the training process
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


<ipython-input-15-946977a1586b>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
500,0.054500
1000,0.059500
1500,0.037300
2000,0.023200


TrainOutput(global_step=2250, training_loss=0.04195105319552951, metrics={'train_runtime': 155.0331, 'train_samples_per_second': 232.209, 'train_steps_per_second': 14.513, 'total_flos': 244228487059584.0, 'train_loss': 0.04195105319552951, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.10023368149995804,
 'eval_f1': 0.9824737105658488,
 'eval_accuracy': 0.9825,
 'eval_runtime': 2.2787,
 'eval_samples_per_second': 877.679,
 'eval_steps_per_second': 54.855,
 'epoch': 2.0}